In [2]:
import requests as re
import pandas as pd
pd.options.display.max_colwidth = 1000

In [432]:
# Define some functions to extract the movie ratings and to cast data
def extract_metacritic(ratings):
    for rating in ratings:
        if rating['Source'] == 'Metacritic':
            return rating['Value']
    return None


def extract_rottentomatoes(ratings):
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None


def convert_date(released):
    if released == 'N/A':
        return None
    else:
        return pd.to_datetime(released, format='%d %b %Y')


def convert_boxoffice(boxoffice):
    if boxoffice == 'N/A':
        return ''
    else:
        return str(boxoffice)

## __MAIN__ ##

##### Movies data is retrieved from the OMBD API, the providers list is collected from the TMDB API. #####

In [465]:
# Create the TMDB token using the API
# read the documentation on the official website:
# https://developer.themoviedb.org/docs
tmdbAPI_KEY = '<< API-KEY >>'
authentication = re.get('https://api.themoviedb.org/3/authentication/token/new?'
                        'api_key=' + tmdbAPI_KEY).json()

# tmdb authentication
token = authentication['request_token']
token

In [3]:
# Load the titles from the csv
file = pd.read_csv('1920-2023_MoviesDatabase.csv')

In [ ]:
# Get all the movies from the OMDB API by titles and create the Df, \
# read the documentation on the official website: \
# https://www.omdbapi.com/
df = []
url = 'http://www.omdbapi.com/?apikey=<<API-KEY>>='
for title in file['Title']:
    movie = re.get(url+title)
    df.append(movie.json())

mdb = pd.DataFrame(df)
mdb.head()

In [458]:
# Create new and drop unuseful columns, drop missing titles
mdb['Budget'] = ''
trash = ['Year', 'Rated', 'Metascore', 'Type', 'DVD', 'Ratings', 'Website',
         'Response', 'Error', 'totalSeasons', 'Language']

mdb.drop(trash, axis=1, inplace=True)
mdb.reset_index(drop=True, inplace=True)
mdb.dropna(axis=0, subset=['Title'], inplace=True)
mdb.columns

In [457]:
# Apply extraction function
mdb['Rotten Tomatoes Rating'] = mdb['Ratings'].apply(extract_rottentomatoes)
mdb['Rotten Tomatoes Rating'] = mdb['Rotten Tomatoes Rating'].map(
    lambda x: (int(x.split('%')[0]))/10 if x else None)

# Apply extraction function
mdb['Metacritic Rating'] = mdb['Ratings'].apply(extract_metacritic)
mdb['Metacritic Rating'] = mdb['Metacritic Rating'].map(
    lambda x: (int(x.split('/')[0]))/10 if x else None)

# Apply convertion function
mdb['Released'] = mdb['Released'].apply(convert_date)
mdb['Runtime'] = mdb['Runtime'].map(lambda x: x.split('min')[0])

# Apply convertion function
mdb['BoxOffice'] = mdb['BoxOffice'].apply(convert_boxoffice)

# retrieve production and budget columns
for e, i in mdb.iterrows():
    movie_id = i.imdbID
    url = 'https://api.themoviedb.org/3/movie/' + \
        str(movie_id) + '?api_key=' + tmdbAPI_KEY + \
        '&language=en-US'

    resp = re.get(url).json()
    try:
        mdb.loc[e, 'Production'] = resp['production_companies'][0]['name']
    except KeyError:
        mdb.loc[e, 'Production'] = ''
    try:
        mdb.loc[e, 'Budget'] = resp['budget']
    except KeyError:
        mdb.loc[e, 'Budget'] = ''

In [846]:
mdb.to_csv('2023_Movies_DB.csv',
           index=False)

In [622]:
# retrieve the full providers list from the tmdb API and create the Df
prov = []
base_url = 'https://api.themoviedb.org/3/watch/providers/movie?api_key='
watchon = re.get(base_url + tmdbAPI_KEY).json()
for i in watchon['results']:
    prov.append({
        'Provider': i['provider_name'],
        'Provider_id': i['provider_id'],
        'Logo_path': i['logo_path']
        })

providers = pd.DataFrame(prov)
providers

,Provider,Provider_id,Logo_path
0,Apple TV,2,/peURlLlr8jggOwK53fJ5wdQl05y.jpg
1,Google Play Movies,3,/tbEdFQDwx5LEVr8WpSeXQSIirVq.jpg
2,Vudu,7,/21dEscfO8n1tL35k4DANixhffsR.jpg
3,Netflix,8,/t2yyOv40HZeVlLjYsCsPHnWLk4W.jpg
4,Amazon Prime Video,9,/emthp39XA2YScoYL1p0sdbAH2WA.jpg
...,...,...,...
524,Anime Generation Amazon Channel,1895,/4RG2VVVUHyTAXbz67QJaHnmsEvb.jpg
525,Raro Video Amazon Channel,1896,/e5h5gWdP50rNxHn8K6LQujuuME5.jpg
526,MIDNIGHT FACTORY Amazon Channel,1897,/oiK6bFbaFY5lZtMziTJ5UBE9UNO.jpg
527,Amazon miniTV,1898,/tcRSEeMnRxtKHqg7kqF8z5SvUpF.jpg


In [843]:
providers_csv = providers.to_csv('Providers_List.csv')

In [612]:
# retrieve the full states list
states = []
url_regions = ('https://api.themoviedb.org/3/watch/providers/regions?api_key='
               + tmdbAPI_KEY)

regions = re.get(url_regions).json()

for region in regions['results']:
    state = {'State': region['english_name'],
             'Code': region['iso_3166_1']}
    states.append(state)

regions = pd.DataFrame(states)
regions

In [844]:
regions_csv = regions.to_csv('Regions.csv')

In [761]:
# create the complete movie_providers table
cols = ['imdbID', 'id_state', 'id_provider',
        'free', 'rent', 'buy', 'provider_url']

providers_list = []
for e, i in mdb.iterrows():
    p = re.get('https://api.themoviedb.org/3/movie/' + str(i.imdbID)
               + '/watch/providers?api_key=' + tmdbAPI_KEY).json()

    if 'results' in p:
        try:
            for state in p['results']: 
                if 'flatrate' in p['results'][state]:
                    flat = p['results'][state]['flatrate'][0]['provider_name']
                else:
                    flat = ''
                if 'free' in p['results'][state]:
                    free = p['results'][state]['free'][0]['provider_name']
                else:
                    free = ''
                if 'rent' in p['results'][state]:
                    rent = p['results'][state]['rent'][0]['provider_name']
                else:
                    rent = ''
                if 'buy' in p['results'][state]:
                    buy = p['results'][state]['buy'][0]['provider_name']
                else:
                    buy = ''
                if 'link' in p['results'][state]:
                    link = p['results'][state]['link']
                else:
                    link = ''
        except KeyError:
            pass

            providers_list.append({
                'imdbID': i.imdbID,
                'State': state,
                'Flatrate': flat,
                'Free': free,
                'Rent': rent,
                'Buy': buy,
                'Link': link
            })

movies_providers = pd.DataFrame(providers_list)

In [964]:
movies_providers = pd.DataFrame(providers_list)
movies_providers.fillna(value='-')
movies_providers

,imdbID,State,Flatrate,Free,Rent,Buy,Link
0,tt1640484,AD,Netflix,-,-,-,-
1,tt1640484,AE,-,-,Apple TV,Apple TV,-
2,tt1640484,AG,-,-,-,Google Play Movies,-
3,tt1640484,AL,Netflix,-,-,Google Play Movies,-
4,tt1640484,AR,-,-,-,Apple TV,-
...,...,...,...,...,...,...,...
373881,tt7339826,AU,-,-,Amazon Video,Amazon Video,https://www.themoviedb.org/movie/476292/watch?locale=AT
373882,tt2573198,AU,HBO Max,-,Apple TV,Apple TV,https://www.themoviedb.org/movie/490794-kusama-infinity/watch?locale=US
373883,tt0142242,AU,-,-,Apple TV,Apple TV,https://www.themoviedb.org/movie/34433/watch?locale=AR
373884,tt5437928,FR,HBO Max,-,Viaplay,Viaplay,https://www.themoviedb.org/movie/454652-colette/watch?locale=FI


In [845]:
df_csv = movies_providers.to_csv('Movies_Providers.csv')

In [572]:
read_mdb = pd.read_csv('2023_Movies_DB.csv')